In [1]:
import os
import sys
import django
import logging
import itertools
import pandas as pd
import numpy as np
from tqdm import tqdm
from django.db.models import Count
from sklearn.metrics import mean_squared_error
sys.path.insert(0, "/Users/victornguyen/Sites/07.book_management/")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
import django
import time
from main_site.models import Rating
import rs_system.neighborhood_cf_rs as nb_rs

logger = logging.getLogger('Evaluation')

In [2]:
ratings_count = Rating.objects.all().count()
test_data_length = round(ratings_count * 10 /100)
all_ratings_df = pd.DataFrame(list(Rating.objects.all().values()))
logger.debug('{} ratings available'.format(ratings_count))
all_ratings_df.head()

2019-07-02 01:31:57,035 : DEBUG : 145 ratings available


,id,rating,rating_timestamp,title_id,type,user_id
0,1,9.43,2019-06-29 02:44:27.101069,20,calculate,8
1,2,9.43,2019-06-29 02:44:27.102891,119,calculate,8
2,3,4.32,2019-06-29 02:44:27.103880,140,calculate,8
3,4,9.08,2019-06-29 02:44:27.104864,113,calculate,9
4,5,9.96,2019-06-29 02:44:27.105703,14,calculate,10


In [3]:
user_ids = Rating.objects.values('user_id') \
    .annotate(title_count=Count('title_id')) \
    .order_by('-title_count')
title_ids = Rating.objects.values('title_id') \
    .annotate(user_count=Count('user_id')) \
    .order_by('-user_count')
user_ids_df = pd.DataFrame.from_records(user_ids)
title_ids_df = pd.DataFrame.from_records(title_ids)
user_ids_df.head(5)

,title_count,user_id
0,7,39
1,5,48
2,4,26
3,4,45
4,3,23


In [ ]:
# all_ratings_df = pd.merge(all_ratings_df, user_ids_df, left_on='user_id', right_on='user_id')
# all_ratings_df = pd.merge(all_ratings_df, title_ids_df, left_on='title_id', right_on='title_id')
# all_ratings_df.head(5)

In [4]:
count = 0
test_data = list()
for row in tqdm(user_ids_df.itertuples()):
    if row.title_count == 1:
        break
    if row.title_count > 10:
        limit_row = 3
    elif row.title_count > 5:
        limit_row = 2
    else:
        limit_row = 1
    test_data.extend(list(Rating.objects.filter(user_id=row.user_id).order_by('-rating').values()[:limit_row]))
    count +=limit_row
    if count == test_data_length:
        break
test_data_df =  pd.DataFrame(test_data)
test_data_id = list(test_data_df.id)
test_data_id
# [41, 58, 23, 54, 18, 1, 130, 83, 139, 105, 21, 48, 51, 28

0it [00:00, ?it/s]


[44, 41, 58, 23, 54, 18, 1, 130, 83, 139, 105, 21, 48, 51]

In [5]:
train_data_df = pd.DataFrame(list(Rating.objects.exclude(id__in=test_data_id).values()))
rating_df = pd.DataFrame(list(Rating.objects.all().values()))
train_data_df

,id,rating,rating_timestamp,title_id,type,user_id
0,2,9.43,2019-06-29 02:44:27.102891,119,calculate,8
1,3,4.32,2019-06-29 02:44:27.103880,140,calculate,8
2,4,9.08,2019-06-29 02:44:27.104864,113,calculate,9
3,5,9.96,2019-06-29 02:44:27.105703,14,calculate,10
4,6,3.44,2019-06-29 02:44:27.106687,164,calculate,11
5,7,9.52,2019-06-29 02:44:27.107523,143,calculate,13
6,8,4.50,2019-06-29 02:44:27.108536,54,calculate,14
7,9,4.50,2019-06-29 02:44:27.109744,66,calculate,15
8,10,7.86,2019-06-29 02:44:27.111010,88,calculate,16
9,11,4.50,2019-06-29 02:44:27.112248,102,calculate,17


In [ ]:
len(test_data)

In [ ]:
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_squared_error(y_true, y_pred)

In [6]:
train_data_df

,id,rating,rating_timestamp,title_id,type,user_id
0,2,9.43,2019-06-29 02:44:27.102891,119,calculate,8
1,3,4.32,2019-06-29 02:44:27.103880,140,calculate,8
2,4,9.08,2019-06-29 02:44:27.104864,113,calculate,9
3,5,9.96,2019-06-29 02:44:27.105703,14,calculate,10
4,6,3.44,2019-06-29 02:44:27.106687,164,calculate,11
5,7,9.52,2019-06-29 02:44:27.107523,143,calculate,13
6,8,4.50,2019-06-29 02:44:27.108536,54,calculate,14
7,9,4.50,2019-06-29 02:44:27.109744,66,calculate,15
8,10,7.86,2019-06-29 02:44:27.111010,88,calculate,16
9,11,4.50,2019-06-29 02:44:27.112248,102,calculate,17


In [7]:
test_data_df

,id,rating,rating_timestamp,title_id,type,user_id
0,44,8.94,2019-06-29 02:44:27.150746,106,calculate,39
1,41,8.94,2019-06-29 02:44:27.147438,88,calculate,39
2,58,8.84,2019-06-29 02:44:27.165850,21,calculate,48
3,23,4.50,2019-06-29 02:44:27.125920,41,calculate,26
4,54,8.96,2019-06-29 02:44:27.161936,97,calculate,45
5,18,7.62,2019-06-29 02:44:27.120113,30,calculate,23
6,1,9.43,2019-06-29 02:44:27.101069,20,calculate,8
7,130,3.63,2019-06-29 02:44:27.244828,1,calculate,114
8,83,3.67,2019-06-29 02:44:27.193088,40,calculate,68
9,139,8.47,2019-06-29 02:44:27.253515,21,calculate,122


In [10]:
res_nb = nb_rs.RecommendationNB()
rating_matrix, item_rating_matrix = res_nb.get_rating_matrix()
mean_centered_rating_matrix = res_nb.get_mean_centered_ratings_matrix(rating_matrix)
item_mean_centered_rating_matrix = res_nb.get_mean_centered_ratings_matrix(item_rating_matrix)

user_similarity_matrix = res_nb.get_similarity_matrix(rating_matrix)
item_similarity_matrix = res_nb.get_similarity_matrix(item_rating_matrix)

2019-07-02 01:32:56,970 : INFO : Load all rating data
2019-07-02 01:32:56,980 : DEBUG : Creating ratings matrix
145it [00:00, 74367.09it/s]


In [11]:
rating_matrix[8]

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan, 9.08,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  n

In [ ]:
y_true = []
y_pred = []
for row in tqdm(test_data_df.itertuples()):
    y_true.append(float(row.rating))
#     predict_result = res_nb.predict(row.user_id, row.title_id, rating_matrix, mean_centered_rating_matrix, user_similarity_matrix)
    y_pred.append(rating_matrix[row.user_id,row.title_id])
print(y_true)
print(y_pred)